# Input data

In [3]:
from copy import deepcopy as copy
df = pd.read_hdf('/scratch/snx3000/musella/cms_tth_2.hd5')

# select di-lepton events
df2l = copy(df[df.num_leptons == 2])
del df
df = df2l

In [ ]:
# utilities to compute 4-vectors
from pyjlr.utils import *

In [6]:
# compute leptons and jet 4 vectors
for ilep in range(2):
    make_p4(df,'leptons',ilep)
    
for ijet in range(10):
    make_p4(df,'jets',ijet)

In [7]:
df['met_px'] = df['met_pt']*np.cos(df['met_phi'])
df['met_py'] = df['met_pt']*np.sin(df['met_phi'])

In [8]:
# compute m^2 of each dijet pair
for ijet in range(10):
    for jjet in range(ijet,10):
        make_m2(df,"jets",ijet,"jets",jjet)

# Find best Higgs candidates and assign other jets to top

In [18]:
import pyjlr.nuSolutions as nusol

In [19]:
from skhep.math.vectors import LorentzVector

In [268]:
import itertools

# set of 1,2 and 4 jets combinations
onejet = list(range(6))
twojets = list(itertools.combinations(onejet,2))
fourjets = list(itertools.combinations(onejet,4))

twojets2ind ={  cmb:icomb for icomb,cmb in enumerate(twojets)  }
fourjets2ind ={  cmb:icomb for icomb,cmb in enumerate(fourjets)  }

# make all possible (higgs,top) jet combinations 
# (under the hypothesis that both higgs jets are reconstructed)
def make_combinations(df,maxjets=6):
    
    #make event hypothesis
    def mk_hyp(X):
        # find dijet pair colses to Higgs mass
        hcand = twojets[np.argmin(np.abs(np.sqrt(X[1:])-125.))]
        # make pairs of all remining jets
        njets = int(X[0])
        if maxjets is not None: njets = min(njets,maxjets)
        alljets = set(range(njets))
        others = itertools.combinations( alljets - set(hcand), 2)
        # return a list of the form [ (hjet0,hjet1), (topjet0,topjet1) ]
        return list(itertools.product([hcand],others))

    # select njet and M^2(i,j) columns
    cols = ["num_jets"]+["jets_jets_m2_%d%d" % x for x in twojets]
    dijets = df[cols]
    
    # compute all events hypotheses
    return dijets.apply(mk_hyp,raw=True,axis=1) #np.abs(np.sqrt(dijets)-125.).apply(mk_hyp,raw=True,axis=1)


In [269]:
df["jets_cmb"] = make_combinations(df)

In [270]:
df[ ["num_jets","jets_cmb"] ].head()

,num_jets,jets_cmb
17,5.0,"[((1, 5), (0, 2)), ((1, 5), (0, 3)), ((1, 5), ..."
54,5.0,"[((3, 5), (0, 1)), ((3, 5), (0, 2)), ((3, 5), ..."
72,4.0,"[((2, 4), (0, 1)), ((2, 4), (0, 3)), ((2, 4), ..."
81,4.0,"[((1, 4), (0, 2)), ((1, 4), (0, 3)), ((1, 4), ..."
96,4.0,"[((1, 4), (0, 2)), ((1, 4), (0, 3)), ((1, 4), ..."


# Run kinematic fit on top hypotheses

In [271]:
reload(nusol)

from numpy.linalg import LinAlgError

# find neutrino solutions assuming mtop and mW
# given 2 leptons and 2 jets up to 8 solutions per event are possible
# 2 parings x 4 solutions of the bi-quadratic system
def make_nusol(df):
    
    maxjets = max(onejet)+1
    # find neutrions solutions given top hypothesis
    def mk_sol(X):
        comb = X[0]
        X = X[1:].values # for some reason raw option does not convert row into np array if one of the columns is of objet type
        jets = X[:4*maxjets].reshape(-1,4)
        leps = X[4*maxjets:4*(maxjets+2)].reshape(-1,4)
        met = X[4*(maxjets+2):]
        
        # loop over all possible combinations
        sols = []
        for icomb in comb:
            isol = []
            # unpack higgs and top hypotheses
            (hjet0,hjet1),(jet0,jet1) = icomb
            ijet0 = LorentzVector( *(jets[jet0].tolist()) )
            ijet1 = LorentzVector( *(jets[jet1].tolist()) )
            # loop over jet-lepton pairings
            for lep0,lep1 in (0,1),(1,0):
                ilep0 = LorentzVector( *(leps[lep0].tolist()) )
                ilep1 = LorentzVector( *(leps[lep1].tolist()) )
                
                # run kinematic fit
                # pad missing solutions with 0's
                try:
                    nsols = 0
                    for sol in nusol.doubleNeutrinoSolutions( (ijet0,ijet1), (ilep0,ilep1), met).nunu_s:
                        nu0,nu1 = sol
                        nu0 = LorentzVector( *nu0, np.sqrt((nu0**2).sum() ))
                        nu1 = LorentzVector( *nu1, np.sqrt((nu1**2).sum() ))
                        top0 = ijet0+ilep0+nu0
                        top1 = ijet1+ilep1+nu1
                        isol.append( np.array( [top0.e,top0.x,top0.y,top0.z, top1.e,top1.x,top1.y,top1.z] ) )
                        nsols += 1
                    for jsol in range(nsols,4):
                        isol.append(np.zeros((1,8)))
                except LinAlgError:
                    for jsol in range(4):
                        isol.append(np.zeros((1,8)))
                    pass
            sols.append(np.vstack(isol))
            
        if len(sols) == 0: return np.zeros((1,8,1))
        # final tensor shape is (ncomb, nsol, 4-vec components, t/tbar)
        return np.vstack(sols).reshape(len(comb),8,4,2)
        
    cols  = ["jets_cmb"] 
    cols += ["jets_%s_%d" % (feat,jet) for jet in onejet for feat in ["px","py","pz","en"] ]
    cols += ["leptons_%s_%d" % (feat,lep) for feat in ["px","py","pz","en"] for lep in range(2)  ]
    cols += ["met_px","met_py"]
    # print(cols)
    return df[cols].apply(mk_sol,raw=True,axis=1)
    

In [277]:
%time dfnusol = make_nusol(df.iloc[:10000])


/opt/python/3.6.1.1/lib/python3.6/site-packages/scipy/optimize/minpack.py:427: RuntimeWarning: Number of calls to function has reached maxfev = 600.
  warnings.warn(errors[info][0], RuntimeWarning)


CPU times: user 2min 20s, sys: 3min 31s, total: 5min 51s
Wall time: 1min 57s


In [279]:
df.shape

(156512, 270)

In [261]:
iev = 3
len(df["jets_cmb"].iloc[iev]),dfnusol.iloc[iev].shape

(3, (3, 8, 4, 2))